In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from utils import load_data, data_split_fashion, FashionDataset, get_dataloaders
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize, Normalize
from torch.utils.data import DataLoader

In [ ]:
data_path = './fashion_full_upd/'
dataloader_params = {'batch_size': 1,
                     'shuffle': True,
                     'num_workers': 8}
resize_normalize_transform = Compose([Resize((224, 224)), ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
val_split = 0.2
num_epochs = 30

styles_df = load_data(data_path)
dataloaders = get_dataloaders(data_path, val_split, resize_normalize_transform, dataloader_params)

In [ ]:
use_cuda = torch.cuda.is_available()

# classes are weighted in proportion to their frequency to ameliorate the severe class imbalance
sorted_class_counts = np.array(styles_df.groupby(['articleType']).size().sort_values(ascending=False))
class_weights = sorted_class_counts / np.sum(sorted_class_counts)
class_weights = torch.tensor(class_weights,  dtype=torch.float)
if use_cuda:
    class_weights = class_weights.cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
model = torchvision.models.resnet50(pretrained=True)

# reinitialize the fc layer
model.fc = nn.Linear(model.fc.in_features, len(class_weights))
if use_cuda:
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def compute_loss(dataloader, model, criterion, use_cuda):
    total_loss = 0.0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            if use_cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss

def train(dataloader_train, dataloader_val, n_epochs, model, criterion, optimizer, use_cuda):
    train_losses, val_losses = [], []
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(dataloaders['others_val']):
            print(i)
            if use_cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        
        train_loss = train_loss / len(dataloader_train.dataset)
        val_loss = compute_loss(dataloader_val, model, criterion, use_cuda) / len(dataloader_val.dataset)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print('Epoch {}: train loss {}, val loss {}'.format(epoch + 1, train_loss, val_loss))
    return train_losses, val_losses, model

In [ ]:
train_losses, val_losses, model = train(dataloaders['top20_train'], 
                                        dataloaders['top20_val'], 
                                        num_epochs, 
                                        model, 
                                        criterion, 
                                        optimizer, 
                                        use_cuda)